# One-Class Classification with Tweet Embeddings

In [1]:
# import modules
import numpy as np
import pandas as pd
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.base import TransformerMixin
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
import re
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer

## Load data

In [2]:
xls = pd.ExcelFile('../../../ForLastExpt.xlsx')

In [3]:
class1_train = pd.read_excel(xls, 'Class=1 Training')
class1_tune = pd.read_excel(xls, 'Class=1 Validation')
class1_test_pt1 = pd.read_excel(xls, 'Class=1 Test Part-1')
class1_test_pt2 = pd.read_excel(xls, 'Class=1 Test Part-2')
class0_train = pd.read_excel(xls, 'Class=0 Training')
class0_tune = pd.read_excel(xls, 'Class=0 Validation')
class0_test_pt1 = pd.read_excel(xls, 'Class=0 Test Part-1')
class0_test_pt2 = pd.read_excel(xls, 'Class=0 Test Part-2')

In [4]:
# create outlier target values

# class1_train
outlier = []
for i in class1_train['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_train['outlier_target'] = outlier

# class1_tune
outlier = []
for i in class1_tune['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_tune['outlier_target'] = outlier

# class1_test_pt1
outlier = []
for i in class1_test_pt1['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_test_pt1['outlier_target'] = outlier

# class1_test_pt2
outlier = []
for i in class1_test_pt2['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_test_pt2['outlier_target'] = outlier

# class0_train
outlier = []
for i in class0_train['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_train['outlier_target'] = outlier

# class0_tune
outlier = []
for i in class0_tune['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_tune['outlier_target'] = outlier

# class0_test_pt1
outlier = []
for i in class0_test_pt1['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_test_pt1['outlier_target'] = outlier

# class0_test_pt2
outlier = []
for i in class0_test_pt2['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_test_pt2['outlier_target'] = outlier

In [5]:
train_fold1 = class0_train
tune_fold1 = pd.concat([class1_tune, class0_tune, class1_train], ignore_index = True)
test_fold1 = pd.concat([class1_test_pt1, class1_test_pt2, class0_test_pt1, class0_test_pt2], ignore_index = True)

train_fold2 = class0_test_pt1
tune_fold2 = pd.concat([class1_test_pt2, class1_test_pt1, class0_test_pt2], ignore_index = True)
test_fold2 = pd.concat([class1_tune, class0_tune, class1_train, class0_train], ignore_index = True)

In [6]:
class1_train

,Is_Unreliable,Category,Tweet,outlier_target
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...,-1
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government...",-1
2,1,"1, 4, 9",It was only a matter of time before the mainst...,-1
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...,-1
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...,-1
...,...,...,...,...
95,1,"11, 10",EOIR has ordered immigration court staff to re...,-1
96,1,10,President - who said last night that #coronavi...,-1
97,1,11,"Mashhad, NE #Iran Man collapses possibly due t...",-1
98,1,"10, 11",Pence caught in lies about insurance covering ...,-1


## Derive text vectors from word embeddings

In [7]:
embeddings = pd.read_csv('word_embed_50.csv', index_col = 0)
embeddings

,Comp 1,Comp 2,Comp 3,Comp 4,Comp 5,Comp 6,Comp 7,Comp 8,Comp 9,Comp 10,...,Comp 41,Comp 42,Comp 43,Comp 44,Comp 45,Comp 46,Comp 47,Comp 48,Comp 49,Comp 50
!,-0.012094,0.000935,0.000976,-0.009518,-0.083408,0.012114,-0.030388,0.020018,0.015333,-0.003408,...,-0.028262,0.036965,-0.017945,-0.020004,0.008269,-0.069600,-0.055546,-0.028868,0.008643,-0.048048
#,0.030090,-0.011546,-0.027933,0.004141,-0.020924,-0.011667,0.028737,0.025169,0.036350,0.117241,...,-0.025798,-0.027569,0.047431,0.084039,0.044840,-0.010098,-0.050790,0.018669,-0.002559,0.039910
(,-0.017210,0.003451,-0.013769,0.016300,-0.011089,0.059428,0.205738,0.022734,-0.092807,-0.003518,...,0.014060,-0.000427,-0.001940,-0.031249,0.007885,-0.006893,0.013552,-0.103300,0.012017,-0.004168
),-0.015370,0.005453,-0.015889,0.016313,-0.010720,0.060504,0.207101,0.022378,-0.093391,-0.001944,...,0.012235,0.000202,-0.003083,-0.031148,0.006398,-0.006262,0.014583,-0.115351,0.009816,-0.012142
",",0.009391,-0.028855,0.000247,0.023929,-0.018705,-0.001752,-0.002686,0.021136,-0.036904,0.031643,...,0.105091,0.072078,-0.036567,0.075362,0.052916,-0.075958,-0.021310,-0.000011,-0.008623,-0.073998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
‘,-0.091469,-0.038311,-0.017282,0.009905,0.011328,-0.017742,0.040392,-0.018368,0.026076,-0.018067,...,0.004947,-0.021305,-0.020591,0.017108,0.047809,0.010627,0.020588,0.003104,0.014091,-0.015461
’,-0.007426,-0.033304,0.013391,0.049407,-0.040433,-0.105308,-0.007539,-0.007917,0.023227,0.011218,...,-0.004001,0.021299,-0.044304,0.040301,0.057719,-0.007113,-0.048529,-0.034061,-0.135474,-0.025738
“,0.009738,-0.022400,-0.024158,0.014539,-0.011919,0.039461,-0.019486,-0.026052,-0.049054,-0.014180,...,0.012043,-0.008504,0.020894,0.004209,0.007849,0.005359,-0.100265,-0.012615,-0.028282,-0.042011
”,0.005733,-0.021793,-0.021662,0.003647,-0.017282,0.040848,-0.024902,-0.011127,-0.030509,0.000597,...,0.005433,-0.012980,-0.006990,0.009010,-0.000684,0.020456,0.002314,-0.008129,-0.027064,0.014475


In [8]:
# to convert contractions picked up by word_tokenize() into full words
contractions = {
    "n't": 'not',
    "'ve": 'have',
    "'s": 'is', # note that this will include possessive nouns
    'gonna': 'going to',
    'gotta': 'got to',
    "'d": 'would',
    "'ll": 'will',
    "'re": 'are',
    "'m": 'am',
    'wanna': 'want to'
}

# to convert nltk_pos tags to wordnet-compatible PoS tags
def convert_pos_wordnet(tag):
    tag_abbr = tag[0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
                
    if tag_abbr in tag_dict:
        return tag_dict[tag_abbr]

In [9]:
def get_text_vectors(word_embeddings, # pandas dataframe with row index = vocabulary
                     text_list, # list of strings
                     remove_stopwords = True,
                     lowercase = True,
                     lemmatize = True,
                     add_start_end_tokens = True):
    
    lemmatizer = WordNetLemmatizer()
    
    for k in range(len(text_list)):
        text = text_list[k]
        text = re.sub(r'[_~`@$%^&*[\]+=\|}{\"\'<>/]+', '', text)
        text_vec = np.zeros(word_embeddings.shape[1])
        words = word_tokenize(text)
        tracker = 0 # to track whether we've encountered a word for which we have an embedding (in each tweet)
        
        if remove_stopwords:
            clean_words = []
            for word in words:
                if word.lower() not in set(stopwords.words('english')):
                    clean_words.append(word)
            words = clean_words

        if lowercase:
            clean_words = []
            for word in words:
                clean_words.append(word.lower())

            words = clean_words

        if lemmatize:
            clean_words = []
            for word in words:
                PoS_tag = pos_tag([word])[0][1]

                # to change contractions to full word form
                if word in contractions:
                    word = contractions[word]

                if PoS_tag[0].upper() in 'JNVR':
                    word = lemmatizer.lemmatize(word, convert_pos_wordnet(PoS_tag))
                else:
                    word = lemmatizer.lemmatize(word)

                clean_words.append(word)

            words = clean_words

        if add_start_end_tokens:
            words = ['<START>'] + words + ['<END>']
        
        for i in range(len(words)):
            word = words[i]
            if word in word_embeddings.index.tolist():
                word_embed_vec = word_embeddings.loc[word].values
                if tracker == 0:
                    text_matrix = word_embed_vec
                else:
                    text_matrix = np.vstack((text_matrix, word_embed_vec))
                    
                # only increment if we have come across a word in the embeddings dictionary
                tracker += 1
                    
        for j in range(len(text_vec)):
            text_vec[j] = text_matrix[:,j].mean()
            
        if k == 0:
            full_matrix = text_vec
        else:
            full_matrix = np.vstack((full_matrix, text_vec))
            
    return full_matrix

In [10]:
train_fold1_X = get_text_vectors(embeddings, train_fold1['Tweet'])
train_fold1_y = train_fold1['outlier_target']

tune_fold1_X = get_text_vectors(embeddings, tune_fold1['Tweet'])
tune_fold1_y = tune_fold1['outlier_target']

test_fold1_X = get_text_vectors(embeddings, test_fold1['Tweet'])
test_fold1_y = test_fold1['outlier_target']

train_fold2_X = get_text_vectors(embeddings, train_fold2['Tweet'])
train_fold2_y = train_fold2['outlier_target']

tune_fold2_X = get_text_vectors(embeddings, tune_fold2['Tweet'])
tune_fold2_y = tune_fold2['outlier_target']

test_fold2_X = get_text_vectors(embeddings, test_fold2['Tweet'])
test_fold2_y = test_fold2['outlier_target']

## Binary classification: two-fold CV

In [11]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

In [12]:
# SVC hyperparams to optimize
kernel = ['rbf', 'linear', 'poly', 'sigmoid']

### Fold 1

In [16]:
# training & tuning
models = []
tune_f1 = []
for ker in kernel:
    svc = OneClassSVM(kernel = ker)
    svc.fit(train_fold1_X, train_fold1_y)
    models.append(svc)
    tune_predict = svc.predict(tune_fold1_X)
    f1 = f1_score(tune_fold1_y, tune_predict, average = 'macro')
    tune_f1.append(f1)
        
opt_model = models[tune_f1.index(max(tune_f1))]
opt_model

OneClassSVM(kernel='poly')

In [17]:
tune_f1

[0.572228823629265, 0.7515527950310559, 0.7831727205337287, 0.7458823529411764]

In [23]:
# testing
test_scores = dict()
test_predict = opt_model.predict(test_fold1_X)
test_predict_proba = opt_model.decision_function(test_fold1_X)
test_scores['auc'] = [roc_auc_score(test_fold1_y, test_predict_proba, average = 'macro')]
test_scores['accuracy'] = [accuracy_score(test_fold1_y, test_predict)]
test_scores['recall'] = [recall_score(test_fold1_y, test_predict, average = 'macro')]
test_scores['precision'] = [precision_score(test_fold1_y, test_predict, average = 'macro')]
test_scores['f1'] = [f1_score(test_fold1_y, test_predict, average = 'macro')]
test_scores

{'auc': [0.7590816326530612],
 'accuracy': [0.6571428571428571],
 'recall': [0.6571428571428571],
 'precision': [0.6925],
 'f1': [0.6406417112299465]}

### Fold 2

In [24]:
# training & tuning
models = []
tune_f1 = []
for ker in kernel:
    svc = OneClassSVM(kernel = ker)
    svc.fit(train_fold2_X, train_fold2_y)
    models.append(svc)
    tune_predict = svc.predict(tune_fold2_X)
    f1 = f1_score(tune_fold2_y, tune_predict, average = 'macro')
    tune_f1.append(f1)
        
opt_model = models[tune_f1.index(max(tune_f1))]
opt_model

OneClassSVM(kernel='linear')

In [25]:
tune_f1

[0.585055021530164, 0.6180555555555556, 0.6145226355894218, 0.6133836093742542]

In [26]:
# testing
test_predict = opt_model.predict(test_fold2_X)
test_predict_proba = opt_model.decision_function(test_fold2_X)
test_scores['auc'].append(roc_auc_score(test_fold2_y, test_predict_proba, average = 'macro'))
test_scores['accuracy'].append(accuracy_score(test_fold2_y, test_predict))
test_scores['recall'].append(recall_score(test_fold2_y, test_predict, average = 'macro'))
test_scores['precision'].append(precision_score(test_fold2_y, test_predict, average = 'macro'))
test_scores['f1'].append(f1_score(test_fold2_y, test_predict))
test_scores

{'auc': [0.7590816326530612, 0.7688265306122449],
 'accuracy': [0.6571428571428571, 0.6785714285714286],
 'recall': [0.6571428571428571, 0.6785714285714286],
 'precision': [0.6925, 0.698143115942029],
 'f1': [0.6406417112299465, 0.6186440677966102]}

### Average results

In [27]:
sum(test_scores['auc'])/2

0.7639540816326531

In [28]:
sum(test_scores['accuracy'])/2

0.6678571428571429

In [29]:
sum(test_scores['recall'])/2

0.6678571428571429

In [30]:
sum(test_scores['precision'])/2

0.6953215579710146

In [31]:
sum(test_scores['f1'])/2

0.6296428895132784